In [81]:
import pandas as pd
import redis
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
import requests
import json
import time

In [ ]:
KERAS_REST_API_URL = "http://40.113.111.0:5000/predict"
r = redis.Redis(host='localhost', port=6379, db=0)
sensordict = {3:"acc_", 4:"gyro_", 5:"mag_"}
sc = pickle.load(open("data/sc.pkl", "rb"))

In [70]:
def get_df():
    dfs = []
    for n in [3,4,5]:
        df = pd.read_msgpack(r.get("sensor"+str(n)+"_ml"))
        dfs.append(df.iloc[:,1:])
    mldf = pd.concat(dfs, axis=1, ignore_index=True)
    mldf.columns = [sensordict[n]+m for n in [3,4,5] for m in ["x", "y","z"]]
    mldf.drop("mag_z", axis=1, inplace=True)
    df = pd.DataFrame(sc.transform(mldf.values))
    return df

In [93]:
while True:
    start = time.time()
    try:
        x = get_df()
    except Exception as e:
        print(e)
        continue
    payload = {"input": x.to_msgpack()}
    # submit the request
    resp = requests.post(KERAS_REST_API_URL, files=payload).json()
    print("got response in "+str(time.time()-start))
    print(resp)
    # ensure the request was sucessful
    if resp["success"]:
        print(resp["squat_prob"])
        r.set("squat_prob", resp["squat_prob"])
    # otherwise, the request failed
    else:
        print("Request failed")
    time.sleep(3)

got response in 0.064208984375
{'squat_prob': '0.00012144653', 'success': True}
0.00012144653
got response in 0.05453753471374512
{'squat_prob': '0.00014244884', 'success': True}
0.00014244884
got response in 0.05614733695983887
{'squat_prob': '0.00012074425', 'success': True}
0.00012074425
got response in 0.052623748779296875
{'squat_prob': '0.0013087752', 'success': True}
0.0013087752
got response in 0.05756402015686035
{'squat_prob': '0.07437233', 'success': True}
0.07437233
got response in 0.05565619468688965
{'squat_prob': '0.012498395', 'success': True}
0.012498395
got response in 0.05972409248352051
{'squat_prob': '0.98699594', 'success': True}
0.98699594
got response in 0.059061288833618164
{'squat_prob': '0.0023879043', 'success': True}
0.0023879043
got response in 0.05185818672180176
{'squat_prob': '0.9807804', 'success': True}
0.9807804
got response in 0.05289459228515625
{'squat_prob': '0.0026858451', 'success': True}
0.0026858451
got response in 0.056166648864746094
{'squa

KeyboardInterrupt: 

In [92]:
r.set("squat_prob", 0.00013)

True